In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Set dataset directory
dataset_dir = 'train'

# Function to add padding to the image
def add_padding(img):
    desired_size = 150
    old_size = img.shape[:2]  # (height, width)
    
    ratio = float(desired_size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    
    img = tf.image.resize(img, new_size)
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    
    new_img = tf.image.pad_to_bounding_box(img, top, left, desired_size, desired_size)
    
    return new_img

# Create data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
   
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    directory=dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    directory=dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load the MobileNetV2 model pre-trained on ImageNet, excluding the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the base model layers
base_model.trainable = False

# Create the new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True

# Compile the model again for fine-tuning
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()


In [ ]:
model.save('new_skin_tone_model.h5')
class_indices = train_generator.class_indices
import json
with open('class_indices.json', 'w') as json_file:
    json.dump(class_indices, json_file)

with open('class_indices.json', 'r') as json_file:
    class_indices = json.load(json_file)

from tensorflow.keras.models import load_model

loaded_model = load_model('new_skin_tone_model.h5')


In [ ]:
class_indices = train_generator.class_indices
with open('class_indices.json', 'w') as json_file:
    json.dump(class_indices, json_file)

with open('class_indices.json', 'r') as json_file:
    class_indices = json.load(json_file)

from tensorflow.keras.models import load_model

loaded_model = load_model('new_skin_tone_model.h5')


In [5]:
import random

# Define expanded clothing colors for each skin tone
colors_for_White = [
    "Light Blue", "Peach", "Lavender", "Mint Green", "Soft Pink", 
    "Coral", "Turquoise", "Ivory", "Soft Yellow", "Baby Blue",
    "Champagne", "Blush", "Pale Green", "Light Grey", "Powder Blue"
]

colors_for_Brown = [
    "Emerald Green", "Warm Beige", "Burnt Orange", "Deep Red", "Royal Blue",
    "Mustard Yellow", "Teal", "Chocolate Brown", "Terracotta", "Copper",
    "Olive Green", "Magenta", "Plum", "Saffron", "Navy Blue"
]

colors_for_Black = [
    "Bright Yellow", "Vibrant Red", "White", "Electric Blue", "Hot Pink",
    "Cobalt Blue", "Silver", "Fuchsia", "Golden Yellow", "Crimson",
    "Aqua", "Orange", "Lime Green", "Scarlet", "Purple"
]

# Mapping from skin tone to clothing colors
skin_tone_to_colors = {
    "White": colors_for_White,
    "Brown": colors_for_Brown,
    "Black": colors_for_Black
}


In [ ]:
class_indices = train_generator.class_indices  # Get the class indices from the training generator
print("Class Indices:", class_indices)  # Print for debugging


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import random

def predict_skin_tone_and_colors(img_path, model, class_indices, skin_tone_to_colors):
    try:
        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(150, 150))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0
        
        # Predict skin tone
        prediction = model.predict(img_array)
        class_idx = np.argmax(prediction, axis=1)[0]
        
        index_to_label = {v: k for k, v in class_indices.items()}
        skin_tone = index_to_label[class_idx]
        
        # Recommend clothing colors based on skin tone
        recommended_colors = random.sample(skin_tone_to_colors[skin_tone], 2)
        
        return skin_tone, recommended_colors
    except Exception as e:
        print(f"Error: {e}")
        return None, None

# Example usage
# image_path example C:\Users\project\Downloads\t9.jpg (Must be jpg)
image_path = r"ImagePath"
print(image_path)
skin_tone, recommended_colors = predict_skin_tone_and_colors(image_path, loaded_model, class_indices, skin_tone_to_colors)

if skin_tone and recommended_colors:
    print(f"Detected skin tone: {skin_tone}")
    print(f"Recommended colors: {recommended_colors}")